In [2]:
from IPython.display import HTML
import jieba

class InvertSearch():
    '''
    InvertSearch() class convert forward seach to backword seach
    '''
    def __init__(self, doc_file):
        '''
        init term, doc_id, doc_list from doc_file
        '''
        self.terms = dict()
        self.doc_id = 0
        self.doc_list = []
        
        #read doc_file
        with open(doc_file, 'r') as f:
            data= f.readlines()
            data = set(data)
            f.close()
        
        #update doc_listselfand doc_id, terms
        for doc in data:
            self.insert(doc)

    def insert(self, doc):
        '''
        insert doc into doc_list return doc_id
        '''
        self.doc_list.append(doc)
        for term in list(jieba.cut(doc)):
            if term not in self.terms:
                self.terms[term] = set([self.doc_id])
            self.terms[term].add(self.doc_id)
        self.doc_id += 1
        return self.doc_id - 1
    
    def find(self, keywords):
        '''
        find keywords in terms reuturn each term set result
        '''
        result = None
        for word in list(jieba.cut(keywords)):
            if result is None:
                result = self.terms.get(word, set())
            else:
                result = result & self.terms.get(word, set())
        if result is None:
            result = set()
        return result
    
    def convert(self, keywords):
        '''
        convert keywords into set expression
        '''
        result = ''
        #convet keywords to standard match format
        keywords_ = keywords.split()
        keywords = ' '.join(keywords_)
        word = list(jieba.cut(keywords))
        print('|'.join(word))
        sign = ['(', ')', 'and', 'AND', 'or', 'OR', 'not', 'NOT', ' ']
        id = 0
        num = len(word)
        while id < num:
            if word[id] == '(' or word[id] == ')' or word[id] == ' ':
                result += word[id]
            elif word[id] == 'and' or word[id] == 'AND':
                result += '&'
            elif word[id] == 'or' or word[id] == 'OR':
                result += '|'
            elif word[id] == 'not' or word[id] == 'NOT'or word[id] == '-':
                result += '-'
            elif id + 1 < num and word[id+1] not in(' ', ')'):
                result +=  'self.find("{}")'.format(word[id]) + ' & '
            elif id + 2< num and word[id+2] not in sign:
                result += 'self.find("{}")'.format(word[id]) + ' & ' 'self.find("{}")'.format(word[id+2])
                id += 3
                continue
            else:
                result +=  'self.find("{}")'.format(word[id])
            id += 1
        return result
    
    def hightlight(self, doc, keywords):
        '''
        hightlight keywords in doc
        '''
        sign = ['(', ')', 'and', 'AND', 'or', 'OR', 'not', 'NOT', ' ']
        for word in list(jieba.cut(keywords)):
            if word not in sign:
                doc = doc.replace(word, '<span style="color:red">{}</span>'.format(word))
        return doc

    def search(self, keywords):
        '''
        searh keywords in doc_list by invert search
        '''
        result = []
        result_ = self.convert(keywords)
        print(result_)
        print(eval(result_))
        for id in eval(result_):
            result.append(self.hightlight(self.doc_list[id], keywords))
        return result

searcher = InvertSearch('titles.txt')
#keywords = '苹果模式 and (三星 or 华为)'
#keywords = '苹果   模式 and (三星 or 华为)'
keywords = '华为'
# keywords = '苹果 芯片 - 华为'
for doc in searcher.search(keywords):
    display(HTML(doc))

华为
self.find("华为")
{320, 385, 448, 165, 485, 456, 363, 173, 242, 407, 281, 27, 157}


In [3]:
searcher = InvertSearch('titles.txt')
#keywords = '苹果模式 and (三星 or 华为)'
#keywords = '苹果   模式 and (三星 or 华为)'
keywords = '华为 not (苹果模式 or 苹果 芯片)'
for doc in searcher.search(keywords):
    display(HTML(doc))

华为| |not| |(|苹果|模式| |or| |苹果| |芯片|)
self.find("华为") - (self.find("苹果") & self.find("模式") | self.find("苹果") & self.find("芯片"))
{320, 385, 448, 165, 485, 363, 173, 242, 407, 281, 27}
